In [ ]:


import sys
# import modules in other directories
sys.path.append('../Utils/')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature



import trax_util as trx
import ibtracs as IBT
import PlotUtil as Pu


import importlib

importlib.reload( trx )
importlib.reload( IBT )

importlib.reload( Pu )



In [ ]:
power_wind=1. # (1./6.)**(0.11)


print( power_wind )

pd1=trx.readtrx( trx.pdfname(ens=1) , power_wind=power_wind )
pd2=trx.readtrx( trx.pdfname(ens=2) , power_wind=power_wind )
pd3=trx.readtrx( trx.pdfname(ens=3) , power_wind=power_wind )
rc1=trx.readtrx( trx.rcp85fname(sst='sst1') , power_wind=power_wind )
rc2=trx.readtrx( trx.rcp85fname(sst='sst2') , power_wind=power_wind )
rc3=trx.readtrx( trx.rcp85fname(sst='sst3') , power_wind=power_wind )
rc4=trx.readtrx( trx.rcp85fname(sst='sst4') , power_wind=power_wind )
rc5=trx.readtrx( trx.rcp85fname(sst='sst5') , power_wind=power_wind )
rc6=trx.readtrx( trx.rcp85fname(sst='sst6') , power_wind=power_wind )
rc7=trx.readtrx( trx.rcp85fname(sst='sst7') , power_wind=power_wind )
ibt=IBT.readtrx()

In [ ]:
plt.scatter( ibt.wind, ibt.pres, s=.1 )
plt.scatter( pd1.wind, pd1.pres, s=.1 )


In [ ]:


xx,yy = ibt.wind.flatten() , ibt.pres.flatten() 
mask = np.isfinite(xx) & np.isfinite(yy) & (xx>5.) 
x0_clean = xx[mask]
y0_clean = yy[mask]
coeff0 = np.polyfit( x0_clean, y0_clean , 2)  # The '2' is for second degree polynomial
print( coeff0 )

xx,yy = pd1.wind.flatten() , pd1.pres.flatten() 
mask = np.isfinite(xx) & np.isfinite(yy) & (xx>5.) 
x1_clean = xx[mask]
y1_clean = yy[mask]
coeff1 = np.polyfit( x1_clean, y1_clean , 2)  # The '2' is for second degree polynomial
print( coeff1 )





In [ ]:

windCats = np.array( [1.,18.,33.,43.,50.,58.,70.] )
presCats = coeff0[0]* windCats**2 + coeff0[1]* windCats + coeff0[2] 
print( presCats)

In [ ]:
pdf_range = [ (0.,120.)  , (860.,1030.) ] 


#H0,x0,y0=np.histogram2d(ibt.wind.flatten() , ibt.pres.flatten()  , bins=40   ,range=pdf_range )
H0,x0,y0=np.histogram2d(x0_clean  , y0_clean , bins=40   ,range=pdf_range )
H0=H0/np.sum(H0 )
H1,x1,y1=np.histogram2d(pd1.wind.flatten() , pd1.pres.flatten()  , bins=40   ,range=pdf_range )
H1=H1/np.sum(H1 )
H2,x2,y2=np.histogram2d(rc1.wind.flatten() , rc1.pres.flatten()  , bins=40   ,range=pdf_range )
H2=H2/np.sum(H2 )


plt.figure( figsize=(10,8))
levels=np.array([1,2,5,10,20,50,100,200,500,1000])*1.e-5
#ax=fig.add_subplot(1,2,1)
plt.contour(x0[1:],y0[1:],H0.T,levels=levels, colors='black',alpha=0.5 ) #,linewidths=widths) #,cmap=cmap,alpha=.5)
plt.contour(x1[1:],y1[1:],H1.T,levels=levels, colors='blue', alpha=0.5 ) #,linewidths=widths) #,cmap=cmap,alpha=.5)
plt.contour(x2[1:],y2[1:],H2.T,levels=levels, colors='red',alpha=0.5 ) #,linewidths=widths) #,cmap=cmap,alpha=.5)
plt.plot( x0, coeff0[0]* x0**2 + coeff0[1]* x0 + coeff0[2] , color='black' , label='IBTrACS')
plt.plot( x0, coeff1[0]* x0**2 + coeff1[1]* x0 + coeff1[2] , color='blue', label='CESM1 PD' )
plt.plot( [],[]  , color='red', label='CESM1 RCP8.5' )
plt.plot( windCats, presCats, 'o' , color='black' )

catlab=['TD','TS','Cat1','Cat2','Cat3','Cat4','Cat5' ]
for ii in np.arange( len(windCats) ):
    plt.text( windCats[ii]+2 ,presCats[ii],catlab[ii], ha='left' , va='top',bbox= dict( edgecolor='black' ) , fontsize=16)

plt.xlabel('Max Wind Speed ($\mathrm{ms}^{-1}$)',fontsize=20)
plt.ylabel('Central Pressure (hPa)',fontsize=20)
plt.legend()
plt.ylim( 1030.,860. )
plt.xlim( 0,100)

"""
plt.text(point_x, point_y, 'This is a point', fontsize=12, ha='left', va='bottom',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', edgecolor='black', alpha=0.5))
"""


In [ ]:
####
intW_ibt,intP_ibt = trx.intf_by_24hr(ibt.wind,ibt.pres,ibt.year,ibt.month,ibt.day,ibt.hour)
intW_pd1,intP_pd1 = trx.intf_by_24hr(pd1.wind,pd1.pres,pd1.year,pd1.month,pd1.day,pd1.hour)
intW_rc1,intP_rc1 = trx.intf_by_24hr(rc1.wind,rc1.pres,rc1.year,rc1.month,rc1.day,rc1.hour)




In [ ]:
#plt.plot(intW.flatten())
nbins=21
Ho,xo=np.histogram(intW_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1,xp1=np.histogram(intW_pd1.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1,xr1=np.histogram(intW_rc1.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1x,xp1x=np.histogram(24.*pd1.intfW3.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1x,xr1x=np.histogram(24.*rc1.intfW3.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1y,xp1y=np.histogram(24.*pd1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1y,xr1y=np.histogram(24.*rc1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
plt.semilogy(xo[1:],Ho/np.sum(Ho) ,color='black',linestyle='--')
plt.semilogy(xp1[1:],Hp1/np.sum(Hp1) ,color='black',linestyle='--')
#plt.semilogy(xr1[1:],Hr1/np.sum(Hr1) ,color='red',linestyle='--')
#plt.semilogy(xp1x[1:],Hp1x/np.sum(Hp1x) ,color='black')
#plt.semilogy(xr1x[1:],Hr1x/np.sum(Hr1x) ,color='red')
plt.semilogy(xp1y[1:],Hp1y/np.sum(Hp1y) ,color='black')
#plt.semilogy(xr1y[1:],Hr1y/np.sum(Hr1y) ,color='red')
plt.xlim(-50,50)
plt.ylim(5e-5,1)

In [ ]:
#plt.plot(intW.flatten())
### Pressure PDF
nbins=21
HoP,xoP=np.histogram(intP_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1P,xp1P=np.histogram(24.*pd1.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp2P,xp2P=np.histogram(24.*pd2.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp3P,xp3P=np.histogram(24.*pd3.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1P,xr1P=np.histogram(24.*rc1.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr2P,xr2P=np.histogram(24.*rc2.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr3P,xr3P=np.histogram(24.*rc3.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr4P,xr4P=np.histogram(24.*rc4.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr5P,xr5P=np.histogram(24.*rc5.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr6P,xr6P=np.histogram(24.*rc6.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr7P,xr7P=np.histogram(24.*rc7.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
xnutP=(xoP[1:]+xoP[:-1])/2.

### Wind PDF
nbins=21
HoW,xoW=np.histogram(intW_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1W,xp1W=np.histogram(24.*pd1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp2W,xp2W=np.histogram(24.*pd2.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp3W,xp3W=np.histogram(24.*pd3.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1W,xr1W=np.histogram(24.*rc1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr2W,xr2W=np.histogram(24.*rc2.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr3W,xr3W=np.histogram(24.*rc3.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr4W,xr4W=np.histogram(24.*rc4.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr5W,xr5W=np.histogram(24.*rc5.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr6W,xr6W=np.histogram(24.*rc6.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr7W,xr7W=np.histogram(24.*rc7.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
#xnutW= 1.944 * (xoW[1:]+xoW[:-1])/2.  # convert to kts per day
xnutW= (xoW[1:]+xoW[:-1])/2.  # leave m s-1 d-1


##################
# MAKE THE PLOTS
##################

xpanlab=-0.02
ypanlab= 1.1
n=0

nxplo,nyplo=2,1
fig = plt.figure(figsize=(15, 6))

n=n+1
Axes0 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
ax1.semilogy(xnutW,HoW/np.sum(HoW),'o-' ,color='black',linestyle='--')

ax1.semilogy(xnutW,Hp1W/np.sum(Hp1W) ,color='black')
ax1.semilogy(xnutW,Hp2W/np.sum(Hp2W) ,color='black')
ax1.semilogy(xnutW,Hp3W/np.sum(Hp3W) ,color='black')

ax1.semilogy(xnutW,Hr1W/np.sum(Hr1W) ,color='red')
ax1.semilogy(xnutW,Hr2W/np.sum(Hr2W) ,color='red')
ax1.semilogy(xnutW,Hr3W/np.sum(Hr3W) ,color='red')
ax1.semilogy(xnutW,Hr4W/np.sum(Hr4W) ,color='red')
ax1.semilogy(xnutW,Hr5W/np.sum(Hr5W) ,color='red')
ax1.semilogy(xnutW,Hr6W/np.sum(Hr6W) ,color='red')
ax1.semilogy(xnutW,Hr7W/np.sum(Hr7W) ,color='red')
ax1.set_xlim(-100,100)
ax1.set_ylim(5e-5,1)

ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_xlabel( "m s$^{-1}$ d$^{-1}$",fontsize=16)
ax1.tick_params(axis='x', labelsize=16)  # Increase x-axis label size
ax1.tick_params(axis='y', labelsize=16)  # Increase y-axis label size
ax1.set_title( "24 hr maximum wind change", fontsize=24 )

ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

ax1.set_xlim(-45.,45.)

n=n+1
Axes0 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
ax1.semilogy(xnutP,HoP/np.sum(HoP),'o-' ,color='black',linestyle='--',label='IBTrACS')

ax1.semilogy(xnutP,Hp1P/np.sum(Hp1P) ,color='black' , label='Present Day' )
ax1.semilogy(xnutP,Hp2P/np.sum(Hp2P) ,color='black')
ax1.semilogy(xnutP,Hp3P/np.sum(Hp3P) ,color='black')

ax1.semilogy(xnutP,Hr1P/np.sum(Hr1P) ,color='red', label='2070-2100' )
ax1.semilogy(xnutP,Hr2P/np.sum(Hr2P) ,color='red')
ax1.semilogy(xnutP,Hr3P/np.sum(Hr3P) ,color='red')
ax1.semilogy(xnutP,Hr4P/np.sum(Hr4P) ,color='red')
ax1.semilogy(xnutP,Hr5P/np.sum(Hr5P) ,color='red')
ax1.semilogy(xnutP,Hr6P/np.sum(Hr6P) ,color='red')
ax1.semilogy(xnutP,Hr7P/np.sum(Hr7P) ,color='red')
ax1.set_xlim(-100,100)
ax1.set_ylim(5e-5,1)

ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_xlabel( "hPa d$^{-1}$",fontsize=16)
ax1.tick_params(axis='x', labelsize=16)  # Increase x-axis label size
ax1.tick_params(axis='y', labelsize=16)  # Increase y-axis label size
ax1.legend(fontsize=18)
ax1.set_title( "24 hr Central pressure change", fontsize=24 )

ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

Pu.save_link(png_filename = "W-P-intfXn.png")
##print(1.944*xo)

In [ ]:
print(np.sum(HoW))
print(np.shape(intW_ibt.flatten()))

In [ ]:
plt.plot( intW_ibt.flatten())

In [ ]:
goo=np.where( np.isfinite(intW_ibt.flatten())==True)
print(np.shape(goo))

In [ ]:
#plt.plot(intW.flatten())
### Pressure PDF
nbins=21
HoP,xoP=np.histogram(intP_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1P,xp1P=np.histogram(24.*pd1.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp2P,xp2P=np.histogram(24.*pd2.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp3P,xp3P=np.histogram(24.*pd3.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1P,xr1P=np.histogram(24.*rc1.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr2P,xr2P=np.histogram(24.*rc2.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr3P,xr3P=np.histogram(24.*rc3.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr4P,xr4P=np.histogram(24.*rc4.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr5P,xr5P=np.histogram(24.*rc5.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr6P,xr6P=np.histogram(24.*rc6.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr7P,xr7P=np.histogram(24.*rc7.intfP24.flatten() , bins=nbins   ,range=(-100.,100.) )
xnutP=(xoP[1:]+xoP[:-1])/2.

### Wind PDF
nbins=21
HoW,xoW=np.histogram(intW_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp1W,xp1W=np.histogram(24.*pd1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp2W,xp2W=np.histogram(24.*pd2.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hp3W,xp3W=np.histogram(24.*pd3.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr1W,xr1W=np.histogram(24.*rc1.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr2W,xr2W=np.histogram(24.*rc2.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr3W,xr3W=np.histogram(24.*rc3.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr4W,xr4W=np.histogram(24.*rc4.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr5W,xr5W=np.histogram(24.*rc5.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr6W,xr6W=np.histogram(24.*rc6.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
Hr7W,xr7W=np.histogram(24.*rc7.intfW24.flatten() , bins=nbins   ,range=(-100.,100.) )
xnutW= 1.944 * (xoW[1:]+xoW[:-1])/2.  # convert to kts per day
#xnutW= (xoW[1:]+xoW[:-1])/2.  # leave m s-1 d-1


##################
# MAKE THE PLOTS
##################

xpanlab=-0.02
ypanlab= 1.1
n=0

nxplo,nyplo=2,1
fig = plt.figure(figsize=(15, 6))

n=n+1
Axes0 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
ax1.semilogy(xnutW,HoW/np.sum(HoW),'o-' ,color='black',linestyle='--')

ax1.semilogy(xnutW,Hp1W/np.sum(Hp1W) ,color='black')
ax1.semilogy(xnutW,Hp2W/np.sum(Hp2W) ,color='black')
ax1.semilogy(xnutW,Hp3W/np.sum(Hp3W) ,color='black')

ax1.semilogy(xnutW,Hr1W/np.sum(Hr1W) ,color='red')
ax1.semilogy(xnutW,Hr2W/np.sum(Hr2W) ,color='red')
ax1.semilogy(xnutW,Hr3W/np.sum(Hr3W) ,color='red')
ax1.semilogy(xnutW,Hr4W/np.sum(Hr4W) ,color='red')
ax1.semilogy(xnutW,Hr5W/np.sum(Hr5W) ,color='red')
ax1.semilogy(xnutW,Hr6W/np.sum(Hr6W) ,color='red')
ax1.semilogy(xnutW,Hr7W/np.sum(Hr7W) ,color='red')
ax1.set_xlim(-100,100)
ax1.set_ylim(5e-5,1)

ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_xlabel( "kts d$^{-1}$",fontsize=16)
ax1.tick_params(axis='x', labelsize=16)  # Increase x-axis label size
ax1.tick_params(axis='y', labelsize=16)  # Increase y-axis label size
ax1.set_title( "24 hr maximum wind change", fontsize=24 )

ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=n+1
Axes0 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
ax1.semilogy(xnutP,HoP/np.sum(HoP),'o-' ,color='black',linestyle='--',label='IBTrACS')

ax1.semilogy(xnutP,Hp1P/np.sum(Hp1P) ,color='black' , label='Present Day' )
ax1.semilogy(xnutP,Hp2P/np.sum(Hp2P) ,color='black')
ax1.semilogy(xnutP,Hp3P/np.sum(Hp3P) ,color='black')

ax1.semilogy(xnutP,Hr1P/np.sum(Hr1P) ,color='red', label='2070-2100' )
ax1.semilogy(xnutP,Hr2P/np.sum(Hr2P) ,color='red')
ax1.semilogy(xnutP,Hr3P/np.sum(Hr3P) ,color='red')
ax1.semilogy(xnutP,Hr4P/np.sum(Hr4P) ,color='red')
ax1.semilogy(xnutP,Hr5P/np.sum(Hr5P) ,color='red')
ax1.semilogy(xnutP,Hr6P/np.sum(Hr6P) ,color='red')
ax1.semilogy(xnutP,Hr7P/np.sum(Hr7P) ,color='red')
ax1.set_xlim(-100,100)
ax1.set_ylim(5e-5,1)

ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_xlabel( "hPa d$^{-1}$",fontsize=16)
ax1.tick_params(axis='x', labelsize=16)  # Increase x-axis label size
ax1.tick_params(axis='y', labelsize=16)  # Increase y-axis label size
ax1.legend(fontsize=18)
ax1.set_title( "24 hr Central pressure change", fontsize=24 )

ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

Pu.save_link(png_filename = "W-P-intfXn.png")
##print(1.944*xo)

In [ ]:
from matplotlib.colors import LogNorm

pdf_range = [(-100,100),(-100,100.)] 

x = 24.*pd1.intfW24.flatten()
y = 24.*pd1.intfP24.flatten()
Hpd1,X0,Y0=np.histogram2d(x, y, bins=nbins   ,range=pdf_range )
Hpd1=Hpd1/np.sum(Hpd1)

x = intW_ibt.flatten() 
y = intP_ibt.flatten() 
Hoo,X0,Y0=np.histogram2d(x, y, bins=nbins   ,range=pdf_range )
Hoo=Hoo/np.sum(Hoo)



lev=np.array( [0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )/10
print(lev)
XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2


cmap=Pu.step_cmap(levels=lev,cmapN='gist_ncar')
nxplo,nyplo=2,1

fig = plt.figure(figsize=(20, 6))

xr0,xr1= pdf_range[0][0],pdf_range[0][1] # -6,2 #-4,2.7 #-3,3.7

Axes0 = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.pcolor(XX,YY, Hpd1.T , cmap='gist_ncar',
              norm=LogNorm(vmin=0.0002, vmax=0.5) ,alpha=1.0)
Axes0 = Pu.axes_def(n=2,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.pcolor(XX,YY, Hoo.T , cmap='gist_ncar',
              norm=LogNorm(vmin=0.0002, vmax=0.5), alpha=1. )



In [ ]:
quints,quint_limits = pd.qcut( pd.Series(  24.*pd1.intfP24.flatten() ), 5,retbins=True )


In [ ]:
print( np.shape(quints))
print( np.shape( pd1.intfP24.flatten()   ))
print( quint_limits )
n=111111
print( quints[n] , pd1.intfP24.flatten()[n] )

In [ ]:
s=666
#oo=np.where( (ibt.hour[s,:]==0 ) & (np.isnan(ibt.pres[s,:])==False))
oo=np.where( (ibt.hour[s,:]==0 ) & (ibt.wind[s,:]>1000.))
print(np.size(oo[0]))
dp = ( ibt.pres[s,10] - ibt.pres[s,0]  )
poo = np.isnan(ibt.pres[s,:])
print( ibt.lon[s,:] )

In [ ]:
print(np.shape(ibt.hour))
plt.plot(ibt.hour[:,0],'.')

In [ ]:
plt.plot(ibt.hour.flatten()[0:],ibt.pres.flatten()[0:],'.')

In [ ]:
plt.plot(pd1.intfW.flatten(),'.')

plt.xlim(0,1000)

In [ ]:
plt.plot(pd1.intfP.flatten(),'.')
plt.plot(rc1.intfP.flatten(),'.')

#plt.xlim(0,100)
#print(pd1.intfP.flatten()[0:10])

In [ ]:
nbins=50
Hp1,xp1=np.histogram(pd1.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hp2,xp1=np.histogram(pd2.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hp3,xp1=np.histogram(pd3.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr1,xr1=np.histogram(rc1.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr2,xr1=np.histogram(rc2.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr3,xr1=np.histogram(rc3.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr4,xr1=np.histogram(rc4.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr5,xr1=np.histogram(rc5.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr6,xr1=np.histogram(rc6.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr7,xr1=np.histogram(rc7.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )
Ho,xo=np.histogram(ibt.intfP.flatten() , bins=nbins   ,range=(-15.,15.) )

In [ ]:
plt.semilogy(xp1[1:],Hp1/np.sum(Hp1) ,color='black')
plt.semilogy(xp1[1:],Hp2/np.sum(Hp2) ,color='black')
plt.semilogy(xp1[1:],Hp3/np.sum(Hp3) ,color='black')
plt.semilogy(xr1[1:],Hr1/np.sum(Hr1) ,color='red')
plt.semilogy(xr1[1:],Hr2/np.sum(Hr2) ,color='red')
plt.semilogy(xr1[1:],Hr3/np.sum(Hr3) ,color='red')
plt.semilogy(xr1[1:],Hr4/np.sum(Hr4) ,color='red')
plt.semilogy(xr1[1:],Hr5/np.sum(Hr5) ,color='red')
plt.semilogy(xr1[1:],Hr6/np.sum(Hr6) ,color='red')
plt.semilogy(xr1[1:],Hr7/np.sum(Hr7) ,color='red')
#plt.semilogy(xo[1:],Ho/np.sum(Ho) ,color='black',linestyle='--')
plt.xlim(-10,0)


In [ ]:
nbins=100
Hp1,xp1=np.histogram(pd1.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hp2,xp1=np.histogram(pd2.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hp3,xp1=np.histogram(pd3.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr1,xr1=np.histogram(rc1.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr2,xr1=np.histogram(rc2.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr3,xr1=np.histogram(rc3.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr4,xr1=np.histogram(rc4.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr5,xr1=np.histogram(rc5.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr6,xr1=np.histogram(rc6.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Hr7,xr1=np.histogram(rc7.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
#Ho,xo=np.histogram(ibt.intfW.flatten() , bins=nbins   ,range=(-15.,15.) )
Ho,xo=np.histogram(intW_ibt.flatten() , bins=nbins   ,range=(-100.,100.) )

xp1_ = xp1*24.
xr1_ = xr1*24.

In [ ]:

plt.semilogy(xp1_[1:],Hp1/np.sum(Hp1) ,color='black')
plt.semilogy(xp1_[1:],Hp2/np.sum(Hp2) ,color='black')
plt.semilogy(xp1_[1:],Hp3/np.sum(Hp3) ,color='black')
"""
plt.semilogy(xr1_[1:],Hr1/np.sum(Hr1) ,color='red')
plt.semilogy(xr1_[1:],Hr2/np.sum(Hr2) ,color='red')
plt.semilogy(xr1_[1:],Hr3/np.sum(Hr3) ,color='red')
plt.semilogy(xr1_[1:],Hr4/np.sum(Hr4) ,color='red')
plt.semilogy(xr1_[1:],Hr5/np.sum(Hr5) ,color='red')
plt.semilogy(xr1_[1:],Hr6/np.sum(Hr6) ,color='red')
plt.semilogy(xr1_[1:],Hr7/np.sum(Hr7) ,color='red')
"""
plt.semilogy(xo[1:],Ho/np.sum(Ho) ,color='black',linestyle='--')

plt.xlim(-110,110)
plt.ylim(1e-5,1)

In [ ]:
print( np.sum( xp1[1:]*Hp1 )/np.sum(Hp1) )
print( np.sum( xr1[1:]*Hr1 )/np.sum(Hr1) )
print( np.sum( xo[1:]*Ho )/np.sum(Ho) )

In [ ]:
pd1

In [ ]:
basi = IBT.basin_outlines(Mediterranean=False,xNEPacific=True)


In [ ]:
basi[8]

In [ ]:
XY=np.array(basi[8].xy)

In [ ]:
print(XY[:,0])

In [ ]:
pd1 = trx.add_sub_basin( pd1, XY[:,0], XY[:,1], sub_basin_number=1 )

In [ ]:
plt.plot( pd1.sub_basin.flatten(),'.' )

In [ ]:
print(np.shape(pd1.sub_basin))

In [ ]:
foo=np.where( pd1.sub_basin.flatten()==1 )

In [ ]:
plt.plot( pd1.lon.flatten(),pd1.lat.flatten(),'.')
plt.plot( pd1.lon.flatten()[foo[0][:]],pd1.lat.flatten()[foo[0][:]],'.')


In [ ]:
power_wind=1.0 # (1./6.)**(0.11)

bace1,YY,MM =trx.basinACEmonth(wind=pd1.wind,basin=pd1.basin,year=pd1.year,month=pd1.month,power_wind=power_wind)
sbace1,YY,MM=trx.basinACEmonth(wind=pd1.wind,basin=pd1.sub_basin,year=pd1.year,month=pd1.month,power_wind=power_wind)


In [ ]:
plt.scatter( bace1[1,:,:].flatten(),sbace1[1,:,:].flatten())
